In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.text import *
import pandas as pd

In [10]:
! {sys.executable} -m pip install kaggle --upgrade   ## hace enlace con kaggle

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [0]:
mv /root/kaggle.json /root/.kaggle/

In [0]:
! chmod 600 /root/.kaggle/kaggle.json

In [0]:
path='/content/'

In [14]:
! kaggle datasets download -d arkhoshghalb/twitter-sentiment-analysis-hatred-speech  ## descargamos el .zip desde kaggle

  0% 0.00/1.89M [00:00<?, ?B/s]
100% 1.89M/1.89M [00:00<00:00, 61.9MB/s]


In [0]:
! unzip -q -n {path}/twitter-sentiment-analysis-hatred-speech.zip -d {path}   #descomprimimos el zip y lo colocamos en path

In [0]:
train_df = pd.read_csv(path + 'train.csv')

In [17]:
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [0]:
train_df.rename(columns={"label": "label", "tweet": "text"}, inplace=True)

In [0]:
change = {1:'racista/sexista', 0:'no racista/sexista'}

In [0]:
train_df['label'].replace(change, inplace = True)
train_df_2 = train_df[['label','text']]
train_df_2.head()
train_df_2.to_csv('/content/train3.csv', index=False)

In [0]:
#train_df.to_csv('/content/train2.csv', index=False)

In [61]:
data_3 = pd.read_csv('/content/train3.csv')
data_3.head()

,label,text
0,no racista/sexista,@user when a father is dysfunctional and is s...
1,no racista/sexista,@user @user thanks for #lyft credit i can't us...
2,no racista/sexista,bihday your majesty
3,no racista/sexista,#model i love u take with u all the time in ...
4,no racista/sexista,factsguide: society now #motivation


In [90]:
data = TextLMDataBunch.from_csv(path, 'train3.csv')
data

TextLMDataBunch;

Train: LabelList (25569 items)
x: LMTextList
xxbos the greatest # love of all # friday,xxbos just me again , .. ! ! ! i would like to say stay and all # xxunk will be xxunk just for your # optimism .,xxbos be n # healthy how many cares one loses when one decides not to be something but to be someone . â coco chanel,xxbos that 's how we feel ! two weeks until miss illinois and miss indiana ! # xxunk # xxunk ¦,xxbos happy i got my paycheck early and 4 days off ð # xxunk
y: LMLabelList
,,,,
Path: /content;

Valid: LabelList (6393 items)
x: LMTextList
xxbos @user this world is not such a wonderful place anymore . gun violence no empathy and a serious disregard for human life is tâ ¦,xxbos # xxunk meeting of @user followed by # bollywood inspired xxunk # minds & & happy xxunk # xxunk,xxbos such a good day ! xxunk # ascot # races # fun # free # family # girlsâ ¦,xxbos @user this guy look like his brother is @user who 's first wife found him w / another man having sex

In [91]:
data_clas = TextClasDataBunch.from_csv(path, 'train3.csv', vocab=data.train_ds.vocab, bs=32)
data_clas

TextClasDataBunch;

Train: LabelList (25569 items)
x: TextList
xxbos summer # morning # photo # style # summer # kharkiv # kh # kharkivgram # ukraine_blog # ukraineâ ¦,xxbos sometimes it 's best to seat , relax and enjoy a glass of wine to # celebrate yourself . cause they wo n't be an other you . # fathersday,xxbos happy wheels â comic level ! â allow 's play ... # game # let # let & 's # play # video # wheels # videos,xxbos hell with the # xxunk i am all for the # xxunk ! ca nt wait till my device comes home ! # xxunk,xxbos wish i could be there have fun guys # love bitten
y: CategoryList
no racista/sexista,no racista/sexista,no racista/sexista,no racista/sexista,no racista/sexista
Path: /content;

Valid: LabelList (6393 items)
x: TextList
xxbos 1 xxrep 4 . 2 xxrep 4 . 3 .. xxunk # xxunk # xxunk # xxunk,xxbos i 'm so sad i put xxunk in my coffee hoping it would cheer me up . did n't work , but made my mouth happy . # xxunk # leaving # funny,xxbos watch # weddingplanning # when p

In [0]:
learn = None   #resetear a nulo y volver a ponerle un valor
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.3)


In [93]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.010507,4.395732,0.297404,00:43


In [94]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.204877,4.025388,0.339914,00:54


In [0]:
learn.save_encoder('/content/ft_enc')

In [96]:
learn2 = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn2.load_encoder('/content/ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (25569 items)
x: TextList
xxbos summer # morning # photo # style # summer # kharkiv # kh # kharkivgram # ukraine_blog # ukraineâ ¦,xxbos sometimes it 's best to seat , relax and enjoy a glass of wine to # celebrate yourself . cause they wo n't be an other you . # fathersday,xxbos happy wheels â comic level ! â allow 's play ... # game # let # let & 's # play # video # wheels # videos,xxbos hell with the # xxunk i am all for the # xxunk ! ca nt wait till my device comes home ! # xxunk,xxbos wish i could be there have fun guys # love bitten
y: CategoryList
no racista/sexista,no racista/sexista,no racista/sexista,no racista/sexista,no racista/sexista
Path: /content;

Valid: LabelList (6393 items)
x: TextList
xxbos 1 xxrep 4 . 2 xxrep 4 . 3 .. xxunk # xxunk # xxunk # xxunk,xxbos i 'm so sad i put xxunk in my coffee hoping it would cheer me up . did n't work , but made my mouth happy . # xxunk # leaving # funny,xxbos watch # weddingp

In [97]:
learn2.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.163584,0.133071,0.951040,00:39


In [98]:
learn2.predict("We need to love one another")


(Category tensor(0), tensor(0), tensor([0.9952, 0.0048]))

In [99]:
data_clas.classes

['no racista/sexista', 'racista/sexista']